# 🔧 Custom Validation Rules

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artiso-ai/dppvalidator/blob/main/examples/notebooks/02_custom_rules.ipynb)

Learn how to add custom business validation logic to dppvalidator.

This notebook covers:
- Creating custom semantic rules
- Adding industry-specific validations
- Configuring rule severity levels

In [ ]:
!pip install -q dppvalidator

## 📋 Understanding Semantic Rules

dppvalidator uses a three-layer validation architecture:

1. **Schema Layer** - JSON Schema validation
2. **Model Layer** - Pydantic type validation
3. **Semantic Layer** - Business rule validation

Custom rules are added at the semantic layer.

## 🏗️ Creating a Custom Rule

Let's create a rule that validates minimum recycled content for fashion products:

In [ ]:
from typing import Literal


class MinimumRecycledContentRule:
    """Custom rule: Fashion products should have minimum 30% recycled content."""

    rule_id: str = "CUSTOM001"
    description: str = "Fashion products require minimum 30% recycled content"
    severity: Literal["error", "warning", "info"] = "warning"
    suggestion: str = "Increase recycled content to at least 30% for sustainability"
    docs_url: str = "https://artiso-ai.github.io/dppvalidator/rules/custom"

    def __init__(self, min_recycled: float = 0.30):
        self.min_recycled = min_recycled

    def check(self, passport) -> list[tuple[str, str]]:
        """Check recycled content meets minimum threshold."""
        violations = []

        if not passport.credential_subject:
            return violations

        scorecard = passport.credential_subject.circularity_scorecard
        if not scorecard:
            return violations

        recycled = scorecard.recycled_content
        if recycled is not None and recycled < self.min_recycled:
            violations.append(
                (
                    "$.credentialSubject.circularityScorecard.recycledContent",
                    f"Recycled content {recycled:.0%} below minimum {self.min_recycled:.0%}",
                )
            )

        return violations


print("✓ Custom rule defined!")

## 🔌 Using Custom Rules with the Engine

In [ ]:
from dppvalidator.validators import ValidationEngine
from dppvalidator.validators.rules import ALL_RULES
from dppvalidator.validators.semantic import SemanticValidator

# Create custom rule set
custom_rules = ALL_RULES + [MinimumRecycledContentRule(min_recycled=0.30)]

# Create engine with custom semantic validator
engine = ValidationEngine()
engine._semantic_validator = SemanticValidator(rules=custom_rules)

print(f"✓ Engine configured with {len(custom_rules)} rules")

## 🧪 Testing the Custom Rule

In [ ]:
# DPP with low recycled content
low_recycled_dpp = {
    "type": ["DigitalProductPassport", "VerifiableCredential"],
    "@context": [
        "https://www.w3.org/ns/credentials/v2",
        "https://test.uncefact.org/vocabulary/untp/dpp/0.6.1/",
    ],
    "id": "https://example.com/credentials/dpp-001",
    "issuer": {"id": "https://example.com/issuers/001", "name": "Fashion Co."},
    "credentialSubject": {
        "type": ["Product"],
        "id": "https://example.com/products/001",
        "name": "Cotton T-Shirt",
        "circularityScorecard": {
            "type": ["CircularityScorecard"],
            "recycledContent": 0.10,  # Only 10% recycled
            "recyclableContent": 0.95,
        },
    },
}

result = engine.validate(low_recycled_dpp)

print(f"Valid: {result.valid}")
print(f"Warnings: {result.warning_count}")

for warning in result.warnings:
    if warning.code == "CUSTOM001":
        print(f"\n🔔 Custom Rule Triggered:")
        print(f"   {warning.message}")
        if warning.suggestion:
            print(f"   💡 {warning.suggestion}")

## 🎚️ Rule Severity Levels

Rules can have different severity levels:

- **error** - Fails validation
- **warning** - Passes but flags issue
- **info** - Informational only

In [ ]:
# Create a strict version of the rule (error instead of warning)
class StrictRecycledContentRule(MinimumRecycledContentRule):
    rule_id = "CUSTOM002"
    severity: Literal["error", "warning", "info"] = "error"  # Now fails validation


strict_rules = ALL_RULES + [StrictRecycledContentRule(min_recycled=0.30)]
engine._semantic_validator = SemanticValidator(rules=strict_rules)

result = engine.validate(low_recycled_dpp)
print(f"Valid with strict rule: {result.valid}")
print(f"Errors: {result.error_count}")

## 📝 Best Practices for Custom Rules

1. **Use descriptive rule IDs** - Follow a naming convention (e.g., `CUSTOM001`, `TXT001`)
2. **Provide helpful suggestions** - Tell users how to fix the issue
3. **Link to documentation** - Include `docs_url` for detailed explanations
4. **Choose appropriate severity** - Use `error` only for hard requirements
5. **Handle missing data gracefully** - Always check if fields exist before accessing

## 🎉 Next Steps

- **[Batch Processing](03_batch_processing.ipynb)** - Validate multiple DPPs efficiently
- **[Plugin Development](05_plugin_development.ipynb)** - Package rules as reusable plugins